<a href="https://colab.research.google.com/github/JapiKredi/FunctionCalling/blob/main/Working_With_ChatGPT_APIs_Part_II_v0_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Working With ChatGPT APIs | Part-II**

In the previous section, we performed the following tasks:
1. Make API calls to the `Completion()` endpoint
2. Modify the prompts and make them more nuanced to perform complex tasks

In the second part of working with ChatGPT APIs, we will use the `ChatCompletion()` API endpoint to create a simple AI-tutor. Specifically, we will:
3. Create a simple 'AI Tutor' using the `ChatCompletion()` endpoint which assists students with math problems
4. Measure the cost of making API calls via tokens and put guardrails in place to monitor and control costs


## Multi-Turn Conversation using `ChatCompletion()` | Math AI Tutor

In the previous section, we used the `Completion()` endpoint to get responses for various prompts. Let's now use the other endpoint offered by OpenAI - `ChatCompletion()`.

`ChatCompletion()` is used for multi-turn, chat-like conversations. For e.g., say we want to build an AI tutor application which helps students with math homework problems. Let's first define what a good tutor looks like. A good tutor will:
* Not reveal the answer to the student, but rather help the student identify their mistakes by asking questions (probing)
* Provide hints, fill gaps in the student's knowledge required to solve the problem
* Provide feedback to guide the student if they are thinking in the right direction

And this will require a conversation with multiple turns, not a single input-output transaction. This is where `ChatCompletion()` API is useful - the bot can be **contextually aware** and make **long, coherent conversations**.

<hr>

An example (good) conversation may look like this:
* Student: Help me solve the equation x^2 - 5x + 6 = 0
* Tutor: Sure. Which step of the solution have you reached?
* Student: Can you tell me the answer first?
* Tutor: As a tutor, I can help you solve the problem by providing guidance, hints or feedback. But I cannot reveal the answer since it will jeopardize your learning.
* Student: Okay. What should be my first step to solve this equation?
* Tutor: Try to factorize the equation, i.e. break it down in the form (x - a)(x - b) = 0.




Let's now get started with the `ChatCompletion()` API. There are three main roles in the API:
1. **System**: This is an instruction that sets the behaviour of the assistant, e.g. "You are a helpful math tutor", or "You are a helpful advisor for financial analysts"
2. **User**: This role represents the end user using the chatbot
3. **Assistant**: This is the ChatGPT chatbot

An API call looks like as follows. We provide an initial system instruction, a user input and an assistant input to  `messages`, which is a list that contains the entire conversation history. In each subsequent API call, we pass on the entire conversation history.

Notice that we are now using the gpt-3.5-turbo model (it is a cheaper than its other ones in the 3.5 family, more on that later).

**IMPORTANT NOTE**: If you notice the code below, we've installed version 0.28 of the OpenAI library. This is because the Completions API used in this notebook is supported only till v0.28 of the OpenAI API. Due to updates in the API, the `ChatCompletion` model in OpenAI is no longer supported and has since been replaced by `chat.completions` model. You can find this in the new notebook shared on the platform.

In [ ]:
# Install OpenAI library
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# once you mount your google drive, you can read data from your google drive into the colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import openai and set the API key
import openai
# Set up the path containint the OpenAI_API_Key.txt file.
filepath = "/content/drive/MyDrive/GenAI_Course_Master/Course_1_ShopAssistAI/Week_2/Session_1/"

with open(filepath + "OpenAI_API_Key_1.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
# simple API call
chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are an AI tutor that assists school students with math homework problems."},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"}
    ]
)

print(chat_response)

{
  "id": "chatcmpl-8VDEptHvLjBP9HHinvHoAvjP4KVvo",
  "object": "chat.completion",
  "created": 1702449747,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To solve for x, you need to isolate it by dividing both sides of the equation by 3. What is the value of x?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 28,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


In [ ]:
# extract only the text
print(chat_response.choices[0]["message"]["content"])

To solve for x, you need to isolate it by dividing both sides of the equation by 3. What is the value of x?


We can now set up some more complex initial system messages, provide more example conversations, store the progressive responses and pass them on to the API for successive conversations.

In [ ]:
with open(filepath + "AI_tutor_system_message_1.txt", "r") as f:
  system_message = ' '.join(f.readlines())

print(system_message)

You are an AI tutor that assists school students with math homework problems. You never reveal the right answer to the student. You ask probing questions to identify where the student might be needing help, provide hints and guidance, and provide directional feedback to indicate if the student is moving in the right direction.
 
 Do not reveal the correct answer to the student.
 



We should note that the `system_message` has its limitations - while it sets the behavior of the bot to an extent, it may not completely determine the behavior. To solve for this, we can provide some examples to the bot - this is called **few shot prompting**.

### Few-Shot Prompting | Providing Examples
Few-shot prompting is the technique of providing examples of behaviours that we expect from the bot. Let's create a list with certain examples which acts as the `messages` object.

In [ ]:
# list of system message, user and assistant samples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"},
        {"role": "assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "user", "content": "3x = 30"},
        {"role": "assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "user", "content": "x = 10"},
    ]

In [ ]:
# get the chatbot's next response
chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = message_history)

print(chat_response.choices[0]["message"]["content"])

Nice work! You correctly solved for x. Keep in mind that to solve an equation, you want to isolate the variable x by performing the same operation on both sides of the equation. In this case, by adding 9 to both sides, you eliminated the -9 term on the left side. Then, dividing both sides by 3 allowed you to isolate x. Well done! Is there anything else I can help you with?


Let's now build a mini-program where we can input the message as an actual user / student and test our AI tutor. But notice that there is one small problem - the examples we have provided are **examples**, not actual conversations that the chatbot should refer to. To clarify that, we can specify the key `name`to `example_user` and `example_assistant`.

In [ ]:
# list of system message, user and assistant examples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"},
        {"role": "user", "content": "Help me solve the equation x - 10 = 2x"}
    ]

In [ ]:
# get the chatbot's next response
chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = message_history)

print(chat_response.choices[0]["message"]["content"])

Let's start by trying to simplify the equation. If we subtract x from both sides, what would the equation become?


Let's now build a mini program which can take inputs so we can chat with our AI tutor and test it. The system message and some initial examples  are provided in `message_history` already.

Let's  write a program which starts with the initial examples, can run a conversation of length n (we need to stop the program somewhere!), add a field which can take the user's input and append it to the `message_history`.

In [ ]:
inp = input()
print(inp)

How to solve 3X - 7 = 14
How to solve 3X - 7 = 14


In [ ]:
# AI tutor mini program
# Enter "exit" to terminate the program

# One user message + one assistant message is one converation
max_conversations = 20
conversation_length = 0 # initialize

# initialize system message, user and assistant examples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"}]

while conversation_length < max_conversations:
  user_input = input()

  # exit if user enters exit
  if "exit" in user_input.lower():
    print("AI Tutor: Exiting the program!")
    break

  message_history.append({"role": "user", "content": user_input})

  chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = message_history).choices[0]["message"]["content"]

  print("\n", "AI Tutor:")
  print(chat_response)
  print("\n")

  message_history.append({"role": "assistant", "content": chat_response}) # add API response to message history
  conversation_length = conversation_length + 1 # another conversation done


How to solve 3X - 7 = 14

 AI Tutor:
To solve the equation 3x - 7 = 14, we need to isolate the variable x on one side of the equation.

First, let's move the constant term (-7) to the right side by adding 7 to both sides of the equation. What do you get when you add 7 to both sides?


3x - 7 + 7 = 21

 AI Tutor:
Hmm, it seems there was a mistake in your calculation. Let me correct it for you.

When we add 7 to both sides of the equation, we have:

3x - 7 + 7 = 14 + 7

This simplifies to:

3x = 21

Now, to solve for x, we need to isolate it. Can you divide both sides of the equation by 3 and tell me what you get?


x = 7

 AI Tutor:
No, I'm sorry, but that's not correct. Let's go through the steps again.

We have:

3x - 7 = 14

To isolate the variable x, let's add 7 to both sides of the equation:

3x - 7 + 7 = 14 + 7

Which simplifies to:

3x = 21

Now, to solve for x, you need to divide both sides of the equation by 3. What is the value of x?


3x/3 = 21/3

 AI Tutor:
That's correct! W

Thus, we have created a simple AI tutor application - it is an initial version and is likely to make many mistakes. Feel free to test it with examples/tasks of varying complexity levels - you will notice that it makes mistakes, such as revealing the right answer, making factual errors, etc. Be aware that there is no single right solution to this - there are many variables you can play with - better system instructions, more examples (better few-shot prompting), other prompting hacks, and model fine-tuning (if we have enough sample data). We will get back to applying advanced prompt engineering techniques and fine-tuning eventually in the course. For now, attempt the following exercise to observe the mistakes it makes and how nuanced prompt design techniques affect ChatGPT's responses.



> ### Exercise: Challenge the AI Tutor with Complex Tasks and Try to Improve Its Performance
* Challenge the AI tutor with more complex math problems and observe how it responds. You can find some [math problems here](https://www.learncbse.in/extra-questions-for-class-8-maths/). One example you can try is:
 -  "*There is a set of 10 cards numbered [6, 5, 3, 9, 7, 6, 4, 2, 8, 2]. Alice randomly picks one card from the set. What is the probability of the card being greater than 5?*"
* Intentionally provide incorrect facts or analysis to the AI tutor and observe if it corrects the mistake
* Try to modify the program so it can improve on its mistakes - you can modify the system message, provide more examples in `messages`, or do something even more creative (think, you will find some non-obvious solutions)!

## Counting Tokens and Computing API Cost

When you develop and deploy applications, it is important to track and monitor the API costs (since ChatGPT models are charged on a per token basis). There is a Python package called `tiktoken` created by OpenAI to compute the number of tokens used per API call.

Let's first look at tokens - ChatGPT models see text in the form of tokens - it converts natural language (any language -- English, Mandarin, Spanish) into tokens.

For e.g. ChatGPT will tokenize the string `"Nelson, my neighbour, loves building AI applications!"` into `['N','elson',',',' my',' neighbour', ',',
' loves',' building',' AI',' applications','!']`. In this example, there are 7 words (plus punctuations) in the sentence and there are 11 tokens.

<br>

A general rule of thumb is **75 words = ~100 tokens**. For quick reference, it is helpful that one page of Google doc or Microsoft Word is about 500 words (~700 tokens).

![image](https://drive.google.com/uc?export=view&id=1blqyToGFtsVzyHMdtHjcqWskToDT5ia9)


<br>

You can [use the Tokenizer here](https://platform.openai.com/tokenizer) to see the number of tokens any given piece of text corresponds to.

In [ ]:
# Simple API call
chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are an AI tutor that assists school students with math homework problems."},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"}
    ]
)

print(chat_response)

{
  "id": "chatcmpl-8VDHSEkxw7nYwqTmXo64RIIE49YJU",
  "object": "chat.completion",
  "created": 1702449910,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "That's correct! To solve for x, divide both sides of the equation by 3. What is the value of x?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 26,
    "total_tokens": 98
  },
  "system_fingerprint": null
}


In [ ]:
def chat_response_with_num_tokens(messages):

    chat_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
    )

    content = chat_response.choices[0].message["content"]

    token_count = {
    'prompt_tokens':chat_response['usage']['prompt_tokens'],
    'completion_tokens':chat_response['usage']['completion_tokens'],
    'total_tokens':chat_response['usage']['total_tokens'],
    }

    return content, token_count

In [ ]:
messages=[
        {"role": "system", "content": "You are an AI tutor that assists school students with math homework problems."},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"}
      ]

chat_response_with_num_tokens(messages)

("That's correct! Now, to solve for x, divide both sides of the equation by 3. What is the value of x?",
 {'prompt_tokens': 72, 'completion_tokens': 28, 'total_tokens': 100})

In [ ]:
# Lets calculate the number of tokens in this message_history
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"},
        {"role": "user", "content": "Help me solve the equation x - 10 = 2x"}]

chat_response_with_num_tokens(message_history)

("Sure! Let's start by eliminating the variable on the right side of the equation. To do that, we can subtract x from both sides of the equation. What do you get?",
 {'prompt_tokens': 252, 'completion_tokens': 37, 'total_tokens': 289})

### **Cost of API Calls**
Let's now calculate the cost of making API calls. We can see the [cost of various APIs here](https://openai.com/pricing). Our AI tutor program runs on gpt-3.5-turbo the cost for which is $0.002 / 1k tokens.


The number of tokens in the `message_history` above is ~250. But be aware that as the chat progresses, the `message_history` keeps getting longer (since we append `user_response` and `chat_response` to the `message_history` after each conversation).

Let's now modify our mini program to include token and cost calculation so that we can track the usage.

In [ ]:
# AI tutor mini program with cost calculation
# Enter "exit" to terminate the program

# One user message + one assistant message is one converation
max_conversations = 20
conversation_length = 0 # initialize

#Let's also keep a track of tokens this time. Initialise an empty dictionary to count tokens
token_count = {'prompt_tokens':0,
               'completion_tokens':0,
               'total_tokens':0,
              }

# initialize system message, user and assistant examples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"}]

while conversation_length < max_conversations:
  user_input = input()

  # exit if user enters exit
  if "exit" in user_input.lower():
    print("AI Tutor: Exiting the program!")
    break

  message_history.append({"role": "user", "content": user_input})

  chat_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = message_history)

  response = chat_response.choices[0]["message"]["content"]

  token_count['prompt_tokens'] += chat_response['usage']['prompt_tokens']
  token_count['completion_tokens'] += chat_response['usage']['completion_tokens']
  token_count['total_tokens'] += chat_response['usage']['total_tokens']

  print("\n", "AI Tutor:")
  print(response)
  print("\n")

  message_history.append({"role": "assistant", "content": response}) # add API response to message history
  conversation_length = conversation_length + 1 # another conversation done

print(token_count)

How to solve 3x - 7 = 14

 AI Tutor:
To solve the equation 3x - 7 = 14, let's isolate the x variable by moving the constant term, -7, to the right-hand side of the equation. Can you try moving -7 to the other side and see what you get?


3x = 14 + 7

 AI Tutor:
Almost there. Remember, when you move -7 to the right-hand side, it changes sign. Can you try again?


3x = 7 + 14

 AI Tutor:
Great! Now that you have correctly moved the -7 to the right-hand side, you can simplify the expression on the right. What is 7 + 14?


3x = 21

 AI Tutor:
Perfect! Now, to solve for x, you'll need to isolate it by dividing both sides of the equation by 3. Can you go ahead and perform that division?


3x/3 = 21/3

 AI Tutor:
That's correct! When you divide both sides of the equation by 3, what do you get?


x = 7

 AI Tutor:
Great job! You've correctly solved the equation. The solution is x = 7. Well done! Is there anything else I can help you with?


exit
AI Tutor: Exiting the program!
{'prompt_tokens':

In [ ]:
# Initialise cost per tokens
cost_per_1k_tokens = 0.002

# Calculate total cost by multiplying this with the total tokens calculated earlier (and also divide by 1000 since the cost is per 1000 tokens)
conversation_cost = "The approx total cost of this conversation is ${0}".format(cost_per_1k_tokens*token_count['total_tokens']/1000)

# Print the conversation cost
print(conversation_cost)

The approx total cost of this conversation is $0.0049960000000000004


### **Setting API Usage Limits**

It is highly recommended to set API usage limits on your OpenAI account to ensure that you don't exceed your monthly budget. You can [do that easily here](https://platform.openai.com/account/billing/limits).



